In [ ]:
import numpy as np 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt 
import seaborn as sns 

# Librerías para el procesamiento de access logs 
import re 
from parse import parse 
from lars.apache import ApacheSource, COMBINED, ApacheWarning

# Manejo de advertencias del sistema, usada para capturar las líneas que no pueden parsearse por problemas de lars (ApacheWarning)
import warnings

# Configuración de estilo para las gráficas
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Ejecutar antes las celdas de `01_access_log_parsing` o elige una ubicación donde exista un archivo de access logs (Apache Combined Log Format) 
log_file_path = '../data/target/access_log_master.log'
log_prep_path = '../data/target/access_log_master.csv'

## Load Access Log File

In [ ]:
# Cambia a None para cargar todos los datos, o a un número para establecer un límite
n_samples = None 

with open(log_file_path, 'r') as f:
  if n_samples is not None: 
    sample_lines = []
    for _ in range(n_samples):
      line = f.readline()
      
      # si se acaba el archivo antes de alcanzar el número de muestras definidas
      if not line:    
        break 
      sample_lines.append(line.strip())
  else: 
    # Cargar todas las líneas
    sample_lines = [line.strip() for line in f]

print(f"Sample log lines (Total: {len(sample_lines)}):")
for i,line in enumerate(sample_lines[:10], 1):
  print(f"{i:>{len(str(n_samples))}}) {line}") 

## Load and Parsing a Access Log File using `lars` y Regex

In [ ]:
regex = '{ip_client} {ident} {auth_user} [{timestamp}] {request_http} {status:d} {size:d} {referrer} {agent}'

parsed_logs = []
conflicting_logs = {}
problematic_logs = {}

with open(log_file_path, 'r') as f:
  all_lines = f.readlines()

# Lista de campos de URL que se quieren extraer de la clase Row de lars
url_fields = [
  ('request_url_scheme', 'scheme'),
  ('request_url_netloc', 'netloc'),
  ('request_url_path_str', 'path_str'),
  ('request_url_params', 'params'),
  ('request_url_query_str', 'query_str'),
  ('request_url_fragment', 'fragment')
]

# Función para extraer campos de URL 
def extract_url_fields(url_obj):
  result = {}
  for field_name, attr_name in url_fields:
    try:
      value = getattr(url_obj, attr_name, None)
      result[field_name] = value if value else None
    except Exception:
      result[field_name] = None
  return result

with warnings.catch_warnings(record=True) as w:
  warnings.simplefilter("always", ApacheWarning)

  with open(log_file_path) as f:
    with ApacheSource(f, log_format=COMBINED) as source:
      for i, row in enumerate(source, 1):
        try:
          record = {
            "remote_host": row.remote_host,
            "ident": row.ident,
            "remote_user": row.remote_user,
            "time": row.time,
            "request_method": row.request.method if hasattr(row, 'request') and row.request else None,
            "request_protocol": row.request.protocol if hasattr(row, 'request') and row.request else None,
            "status": row.status,
            "size": row.size,
            "req_Referer": row.req_Referer,
            "req_User_agent": row.req_User_agent,
          }
          
          # Extraer campos de URL si existe
          if hasattr(row, 'request') and row.request and row.request.url:
            url_data = extract_url_fields(row.request.url)
            record.update(url_data)
          else:
            # Si no hay URL, establecer todos los campos como None
            record.update({field_name: None for field_name, _ in url_fields})
          
          line_content = all_lines[i-1].strip()
          parsed = parse(regex, line_content)
          if parsed:
            record['request_http'] = parsed.named['request_http']
          else: 
            print(f"Error?: Line: {line_content}")
          
          parsed_logs.append(record)
        except Exception as e:
          conflicting_logs[i] = {
            'error': str(e),
            'line_content': all_lines[i-1].strip() if i <= len(all_lines) else "not available"
          }
  
  for warning in w:
    if issubclass(warning.category, ApacheWarning):
      msg = str(warning.message)
      match = re.search(r'Line (\d+):', msg)
      if match:
        line_num = int(match.group(1))
        if line_num <= len(all_lines):
          problematic_logs[line_num] = {
            'line_content': all_lines[line_num - 1].strip(),
            'warning_message': msg,
            'category': warning.category.__name__,
          }

total_problematic_logs = len(problematic_logs)
total_conflicting_logs = len(conflicting_logs)
total_parsed_logs = len(parsed_logs)
total_logs = total_parsed_logs + total_problematic_logs + total_conflicting_logs

print(f"Total de logs procesados: {total_logs}")
print(f"Problematic Logs: {total_problematic_logs} ({(total_problematic_logs / total_logs * 100 if total_logs != 0 else 0):.2f}%)")
print(f"Conflicting Logs: {total_conflicting_logs} ({(total_conflicting_logs / total_logs * 100 if total_logs != 0 else 0):.2f}%)")
print(f"Parsed Logs: {total_parsed_logs} ({(total_parsed_logs / total_logs * 100 if total_logs != 0 else 0):.2f}%)")

Mientras que `conflicting_logs` son errores técnicos del parser, `problematic_logs` son anomalías en el contenido que pueden indicar actividad maliciosa o intentos de exploración, pero para confirmar que estos logs representan una actividad maliciosa se debe analizar el contenido de estos logs 

In [ ]:
df_parsed = pd.DataFrame(parsed_logs)  
display(df_parsed.head(5))
display(df_parsed.tail(5))

### Análisis de Logs Problemáticos (`problematic_logs`)

In [ ]:
max_idx = max(problematic_logs.keys())

logs = list(problematic_logs.items()) 

for log in logs:
  idx = log[0]
  line_content = log[1]['line_content']
  print(f"{idx:>{len(str(max_idx))}}) {line_content}") 

Presentan:
- Cadenas bytes en formato hexademical en las solicitudes HTTP (tanto en método, protocolo y url)
- No existe Referer, ni User-Agent
- Las pocas solicitudes HTTP (las que no están formadas por bytes en formato hexademical) no presentan método y tienen cadenas muy extrañas o tienen direcciones IP 

In [ ]:
from parse import parse 

pattern = '{ip_client} {ident} {auth_user} [{timestamp}] {request_http} {status:d} {size:d} "{referrer}" "{agent}"'

problematic_parsed_logs = []
for problematic in list(problematic_logs.items()):
  idx = problematic[0]
  line_content = problematic[1]['line_content']
  try: 
    parsed = parse(pattern, line_content.strip())
    problematic_parsed_logs.append(parsed.named)
  except Exception as e:
    print(f"Line {idx}: {line_content}")
    print(f"Error: {e}")

df_problematic = pd.DataFrame(problematic_parsed_logs)
status_distribution = df_problematic['status'].value_counts().sort_index()
display(df_problematic.shape[0])
display(status_distribution)

**Análisis de Distribución de Código de Estado HTTP**:
- `301`: El recurso solicitado tiene una nueva URL permanente 
- `400`: El servidor no puede entender la solicitud debido a una sintaxis inválida 
- `408`: El servidor esperó demasiado tiempo a que el cliente enviara la solicitud completa y cerró la conexión inactiva
- `499`: El cliente cerro la solicitud antes de que el servidor pudiera responder

Para códigos 400, se va revisar la dirección IP de origen y el agente de usuario $\to$ Un alto volumen de errores 400 provenientes de unas pocas IPs o de agentes de usuario inusuales (como herramientas de escaneo) puede indicar actividad maliciosa automatizada. 

Para códigos 408 y 499: Estos códigos suelen ser síntomas de que algo en el servidor no funciona de manera óptima $\to$ Un aumento repentino suele estar relacionado con una alta carga en el servidor, tiempos de respuesta lentos en la aplicación o problemas de red

In [ ]:
# Filtrar logs con los códigos de estado de interés
status_codes_to_check = [400, 408, 499]
filtered_logs = df_problematic[df_problematic['status'].isin(status_codes_to_check)]

# Agrupar por IP para identificar posibles orígenes problemáticos
ip_analysis = filtered_logs.groupby(['status', 'ip_client']).size().reset_index(name='count')
print(ip_analysis.sort_values(by='count', ascending=False).head(20))  # Ver las 20 IPs más activas

# Agrupar por agente de usuario
agent_analysis = filtered_logs.groupby(['status', 'agent']).size().reset_index(name='count')
print(agent_analysis.sort_values(by='count', ascending=False).head(10))  # Ver los 10 agentes más comunes

Estos resultados muestran una **actividad maliciosa**: patrón de cientos de errores 400, concentrados en pocas IPs, cada una con rangos muy diferentes, y con agente vacío, no es un comportamiento normal de un usuario o buscador. 

## Análisis Exploratorio de Datos sobre Access Log Master File

Ya se tienen dos DataFrames (`df_parsed` que son los resultados de parsear correctamente los logs y `df_problematic` son los access logs parseados que presentan un contenido inusual) y se va a analizar ambos como un único DataFrame

In [ ]:
display(df_parsed.head(3))
display(df_problematic.head(2))

La relaciones entre las columnas de los DataFrames `df_parsed` y `df_problematic` son:

| DataFrame `df_parsed`   | DataFrame `df_problematic` |
| ----------------------- | -------------------------- |
| `remote_host`           | `ip_client`                |
| `ident`                 | `ident`                    |
| `remote_user`           | `auth_user`                |
| `time`                  | `timestamp`                |
| `request_method`        | -                          |
| `request_protocol`      | -                          |
| `status`                | `status`                   |
| `size`                  | `size`                     |
| `req_Referer`           | `referrer`                 |
| `req_User_agent`        | `agent`                    |
| `request_url_scheme`    | -                          |
| `request_url_netloc`    | -                          |
| `request_url_path_str`  | -                          |
| `request_url_params`    | -                          |
| `request_url_query_str` | -                          |
| `request_url_fragment`  | -                          |
| `request_http`          | `request_http`             |


Debido a que el DataFrame `df_problematic` muestra una actividad maliciosa pero no se garantiza nada en el DataFrame de `df_parsed`, no se puede asumir que todos los logs de `df_parsed` presentan una actividad maliciosa, por lo que se va a añadir una columna que sea: `malicious_activity_by_parsing`

In [ ]:
# Añadir columna "malicious_activity_by_parsing" a df_parsed y df_problematic 
df_problematic['malicious_activity_by_parsing'] = True 
df_parsed['malicious_activity_by_parsing'] = False 

# Renombrar columnas de df_problematic para que coincidan con df_parsed (según el mapeo proporcionado)
df_problematic = df_problematic.rename(columns={
  'ip_client': 'remote_host',
  'auth_user': 'remote_user',
  'timestamp': 'time',
  'referrer': 'req_Referer',
  'agent': 'req_User_agent',
  # status, size, ident ya tienen el mismo nombre
})

# Añadir columnas faltantes en df_problematic (que existen en df_parsed pero no en df_problematic)
# Estas son las columnas de URL y las relaciones con request
url_columns = [
  'request_url_scheme',
  'request_url_netloc', 
  'request_url_path_str',
  'request_url_params',
  'request_url_query_str',
  'request_url_fragment'
] 
request_columns = [
  'request_method', 
  'request_protocol'
]

# Añadir todas las columnas faltantes con valor None
for col in url_columns + request_columns:
  if col not in df_problematic.columns:
    df_problematic[col] = None 

# Verificar que ambos DataFrames tienen las mismas columnas 
all_columns = set(df_parsed.columns) | set(df_problematic.columns)

# Unificar ambos DataFrames
df = pd.concat([df_parsed, df_problematic], ignore_index=True)
display(df.head(5))
display(df.tail(5))

In [ ]:
# transformar 'time' a datetime (ajustando el formato de Apache) 
def parse_apache_time(time_str):
  "Convierte el formato de tiempo de Apache Access Log a datetime"
  try:
    # Ejemplo de formato: [01/01/2001:00:00:01 -0700]
    return pd.to_datetime(time_str, format='[%d/%b/%Y:%H:%M:%S %z]')
  except: 
    return pd.NaT 

df['time'] = df['time'].apply(parse_apache_time)

# transformar 'size' a numérico (manejando valores no numéricos como '-')
df['size'] = pd.to_numeric(df['size'], errors='coerce').fillna(0)

In [ ]:
print(f"Rango de Tiempo: {df['time'].min()} -> {df['time'].max()}")
print(f"Estadísticas de 'size': Min={ df['size'].min() }, Max={ df['size'].max() }, Mean={ df['size'].mean() }")

In [ ]:
# Mostrar las primeras K filas con los valores 'size' más altos
K = 10
display_cols = [
  'time', 
  'remote_host', 
  'request_method', 
  'request_http', 
  'size', 
  'status', 
  'req_User_agent'
]
# Ordenar por tamaño de forma descendente y tomar las primeras K filas
top_K_largest = df.nlargest(K, 'size')
above_mean = df[df['size'] > df['size'].mean()]
print(f"Logs con tamaño de respuesta > promedio (promedio/mean={df['size'].mean():.2f}): {len(above_mean)} ({(len(above_mean)/len(df))*100:.2f}%)")
display(top_K_largest[display_cols])

In [ ]:
plt.figure(figsize=(15, 6))

# Crear scatter plot con transparencia para manejar muchos puntos
plt.scatter(df['time'], df['size'], alpha=0.6, s=10, color='royalblue')

plt.title('Tamaño de respuestas a lo largo del tiempo', fontsize=14, fontweight='bold')
plt.xlabel('Fecha y Hora', fontsize=12)
plt.ylabel('Tamaño (bytes)', fontsize=12)
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

In [ ]:
plt.figure(figsize=(15,6))

# agrupar por día 
df_daily = df.set_index('time').resample('D').size()
# crear gráfica de barras para peticiones diarias 
bars = plt.bar(df_daily.index, df_daily.values, width=0.8, color='skyblue', edgecolor='navy', alpha=0.8)

plt.title('Número de peticiones por día', fontsize=14, fontweight='bold')
plt.xlabel('Fecha', fontsize=12)
plt.ylabel('Cantidad de peticiones', fontsize=12)
plt.xticks(rotation=45)

for bar in bars:
  height = bar.get_height()
  plt.text(bar.get_x() + bar.get_width()/2, height, f'{int(height)}', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
important_columns = [
  'request_method',
  'status',
  'remote_host',
  'request_http',
  'req_User_agent',
]

# Filtrar solo columnas existentes
existing_cols = [col for col in important_columns if col in df.columns]
n_cols = len(existing_cols)

# Configurar subplots
fig, axes = plt.subplots(n_cols, 1, figsize=(15, 5 * n_cols))
if n_cols == 1:
  axes = [axes]

# Crear gráficas para cada columna
for idx, col in enumerate(existing_cols):
  ax = axes[idx]
  
  # Obtener value counts (top 10 para evitar sobrecarga)
  value_counts = df[col].value_counts().head(10)
  
  # Para columnas con muchos valores únicos, mostrar solo top
  if len(value_counts) > 10:
    others_count = df[col].value_counts().iloc[10:].sum()
    if others_count > 0:
      value_counts['Otros'] = others_count
  
  # Crear gráfica de barras horizontal
  bars = ax.barh(range(len(value_counts)), value_counts.values, color=plt.cm.tab20c(range(len(value_counts))))
  
  ax.set_title(f'Distribución de {col}', fontsize=12, fontweight='bold')
  ax.set_xlabel('Frecuencia', fontsize=10)
  ax.set_yticks(range(len(value_counts)))
  ax.set_yticklabels(value_counts.index, fontsize=9)
  
  # Añadir etiquetas de valores
  total = value_counts.sum()
  for i, v in enumerate(value_counts.values):
    percentage = (v / total) * 100
    ax.text(v + max(value_counts.values) * 0.01, i, f'{v} ({percentage:.1f}%)', va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Gráfica adicional: Códigos de estado HTTP (si existe la columna)
if 'status' in df.columns:
  plt.figure(figsize=(12, 6))
    
  # Agrupar códigos por categoría
  df['status_category'] = df['status'].astype(str).str[0] + '00'
    
  # Contar por categoría
  status_counts = df['status_category'].value_counts()
    
  # Crear gráfico de pastel
  colors = ['#4CAF50', '#2196F3', '#FF9800', '#F44336', '#9C27B0']
  wedges, texts, autotexts = plt.pie(status_counts.values, labels=status_counts.index, autopct='%1.1f%%', colors=colors, startangle=90)
  
  # Mejorar etiquetas
  for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')
  
  plt.title('Distribución de códigos de estado HTTP por categoría', fontsize=14, fontweight='bold')
  plt.axis('equal')
  
  # Añadir leyenda con códigos específicos más comunes
  plt.figure(figsize=(10, 6))
  top_status = df['status'].value_counts().head(10)
  top_status.plot(kind='barh', color='steelblue')
  plt.title('Top 10 códigos de estado HTTP más frecuentes', fontsize=12)
  plt.xlabel('Frecuencia')
  plt.tight_layout()
  plt.show()

In [ ]:
df['url_length'] = df['request_http'].apply(len)
df['url_depth'] = df['request_http'].apply(lambda x : x.count('/'))
df['n_encoded_chars'] = df['request_http'].apply(lambda x : len(re.findall(r'%[0-9A-Fa-f]{2}', x)))
df['n_special_chars'] = df['request_http'].apply(lambda x : len(re.findall(r'[|,;]', x)))

In [ ]:
display(df.shape[0])
display(df.isnull().sum())

In [ ]:
df = df.drop(df.columns[[1, 2, 13]], axis=1)
display(df.shape[0])
display(df.isnull().sum())

In [ ]:
df = df.drop(columns='request_url_fragment', axis=1)
display(df.shape[0])
display(df.isnull().sum())

In [ ]:
columns = [
  'request_url_scheme',
  'request_url_netloc',
  'request_url_path_str',
  'request_url_query_str',
  'req_Referer',
  'req_User_agent',
  'request_method',
  'request_protocol'
]
for col in columns:
  print(f"Valores Únicos de la Columna {col}: {df[col].unique()}")
  print()

- Dentro de `request_method` existen algunos métodos como `DELETE`, analizar si estos 
- 

In [ ]:
# Paso final: guardar el dataset procesado
df.to_csv(log_prep_path, index=False)